# Song Popularity Regression Model
# Authors: 

**Original Dataset Source: https://www.kaggle.com/datasets/yasserh/song-popularity-dataset**

In [1]:
# Libraries

import sklearn
import numpy
import pandas
import matplotlib.pyplot
import seaborn 